In [1]:
print("okay")


okay


In [4]:
%pwd

'/home/sneeraj/medbot/Med-bot---demo'

In [3]:
import os 
os.chdir("../")

In [6]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [15]:
#Extract Data from PDF 

def load_pdf_file(data):
    loader = DirectoryLoader(data,
                             glob="*.pdf",
                             loader_cls=PyPDFLoader)
    documents= loader.load()
    return documents

In [16]:
extracted_data = load_pdf_file(data="Data/")

In [19]:
#extracted_data

In [182]:
#Split the data into Text Chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [183]:
text_chunks = text_split(extracted_data)
print(len(text_chunks))

6973


In [26]:
#text_chunks

In [27]:
from langchain.embeddings import HuggingFaceEmbeddings

In [29]:
#Downloading embeddings from huggingface

def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [32]:
embeddings = download_hugging_face_embeddings()

In [160]:
query_result = embeddings.embed_query("Hello world")
print(len(query_result))

384


In [41]:
#query_result


In [202]:
from dotenv import load_dotenv
load_dotenv()

True

In [203]:
PINECONE_API_KEY=os.environ.get("PINECONE_API_KEY")
GROQ_API_KEY=os.environ.get("GROQ_API_KEY")

In [131]:
from pinecone import Pinecone, ServerlessSpec
import os

pc = Pinecone(api_key = PINECONE_API_KEY)


pc.create_index(
    name = "medbot-index",
    dimension = 384,
    metric = "cosine",
    spec = ServerlessSpec(
        cloud = "aws",
        region = "us-east-1",
    )
)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2025-04', 'x-cloud-trace-context': 'e028dfde0f69644a78950bd36c2c64ab', 'date': 'Wed, 20 Aug 2025 16:29:36 GMT', 'server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [204]:
import os 
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["GROQ_API_KEY"] = GROQ_API_KEY

In [201]:
#embed each chunk and upsert the embeddings into your pinecone index

from langchain_pinecone import PineconeVectorStore
docsearch = PineconeVectorStore.from_documents(
    documents = text_chunks,
    index_name = "medbot-index",
    embedding = embeddings,
)

KeyboardInterrupt: 

In [165]:
# Load Existing index

from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_existing_index(
    index_name="medbot-index",
    embedding = embeddings,
)

In [166]:
docsearch

In [167]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [205]:
retrieved_docs = retriever.invoke("What is acne?")

In [206]:
retrieved_docs

[Document(id='47c417b3-2053-4620-b908-b2b63f3b73fe', metadata={'author': '', 'creationdate': '2017-05-01T10:37:35-07:00', 'creator': '', 'keywords': '', 'moddate': '2017-05-01T10:37:35-07:00', 'page': 425.0, 'page_label': '426', 'producer': 'GPL Ghostscript 9.10', 'source': 'Data/The_GALE_ENCYCLOPEDIA_of_MEDICINE_SECOND.pdf', 'subject': '', 'title': '', 'total_pages': 759.0}, page_content='Corticosteriod —A group of synthetic hormones\nthat are used to prevent or reduce inflammation.\nToxic effects may result from rapid withdrawal after\nprolonged use or from continued use of large doses.\nPatch test—A skin test that is done to identify aller-\ngens. A suspected substance is applied to the skin.\nAfter 24–48 hours, if the area is red and swollen,\nthe test is positive for that substance. If no reaction\noccurs, another substance is applied. This is con-'),
 Document(id='ac2e9df6-747e-43fb-a64a-28de220bbb3a', metadata={'author': '', 'creationdate': '2017-05-01T10:37:35-07:00', 'creator'

In [207]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",
    api_key=GROQ_API_KEY,
    model="llama3-8b-8192",  # You can try mistralai/mixtral-8x7b or others too
)

In [208]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If the context does not help,do your best to answer based on what you know.\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


In [209]:
question_answer_chain = create_stuff_documents_chain(llm,prompt)
rag_chain = create_retrieval_chain(retriever,question_answer_chain)

In [213]:
response = rag_chain.invoke({"input": "What is Contractures"})
print(response["answer"])

According to the provided context, Contractures are the chronic loss of joint motion due to structural changes in non-bony tissue. These non-bony tissues include muscles, ligaments, and tendons.
